### 랭체인
랭체인(LangChain)은 대규모 언어 모델(LLM)을 기반으로 한 어플리케이션 개발을 위한 혁신적인 프레임워크로, 언어 모델을 사용한 다양한 작업을 효과적으로 수행할 수 있게 해주는 도구입니다. 현대적이고 모듈화된 구조를 가진 랭체인은 주로 파이썬과 타입스크립트/자바스크립트 두 가지 버전으로 제공되어, 다양한 환경에서 활용될 수 있습니다.

랭체인은 여러 모듈로 구성되어 있으며, 이 모듈들이 함께 작동하여 언어 모델과 데이터 소스를 효과적으로 연결하고, 애플리케이션의 작업 흐름을 체계적으로 관리합니다.
랭체인은 모듈들 간의 조합을 통해 언어 모델과 데이터 소스를 효과적으로 연결하고, 이를 통해 다양한 작업을 수행합니다. 각 모듈은 모듈화된 추상화 및 구현을 통해 유연성을 제공하며, 사용자는 구성 요소를 조합하여 기존 체인을 맞춤설정하거나 새로운 체인을 생성할 수 있습니다.

### 주요 모듈

1. **모델 I/O (입출력):** 언어 모델과의 상호작용을 관리하며, 프롬프트를 조작하고 언어 모델의 출력에서 정보를 추출합니다.

2. **데이터 연결:** 애플리케이션별 데이터를 로드, 변형, 저장 및 쿼리하기 위한 필수 빌딩 블록을 제공합니다.

3. **체인:** 작업의 호출 순서를 구축하고 체인으로 연결된 다양한 구성 요소를 효율적으로 관리합니다.

4. **에이전트:** 상위 지시문을 받으면 어떤 툴을 사용할지 결정하여 체인을 유연하게 조작합니다.

5. **메모리:** 대화형 시스템이 과거 메시지에 직접 액세스할 수 있도록 해주는 기능을 제공합니다.

6. **콜백:** 체인의 중간 단계를 기록하고 스트리밍하여 로깅, 모니터링 및 기타 작업에 활용됩니다.


### 랭체인 사용 사례

랭체인은 다양한 사용 사례에 적용될 수 있습니다. 이에는 문서에 대한 Q&A, 구조화된 데이터 분석, API 통합, 코드 이해, 에이전트 시뮬레이션, 챗봇 개발, 코드 작성, 데이터 추출, 그래프 데이터 분석, 멀티 모달 출력, 자가 검사, 요약, 태깅 등이 포함됩니다. 또한 랭체인은 다양한 통합을 지원하여 콜백, 챗 모델, 문서 로더, 데이터베이스, 검색 방법, 텍스트 임베딩 모델, 에이전트 툴킷, 툴, 벡터 저장소 등을 활용할 수 있도록 제공됩니다.

### 랭체인 사용법을 위한 챗봇
https://langchain-teacher-lcel.streamlit.app/

### 참조자료
참조자료를 보고 랭체인 연습 튜토리얼을 한글판으로 제작하였습니다.

https://github.com/gkamradt/langchain-tutorials

https://github.com/wikibook/openai-llm


In [37]:
# !pip install langchain
# !pip install openai
# !pip install tiktoken
# !pip install sentence-transformers
# !pip install pypdf
# !pip install faiss-cpu

In [3]:
import os
import getpass

openai_api_key = os.getenv('OPENAI_API_KEY', getpass.getpass("api"))

api ···················································


### Chat Message

가장 많이 사용하는 Chat Messege를 통해서는 OpenAI의 gpt3.5와 gpt4를 불러 올 수 있습니다. 
그렇게 위해서는 System : AI에게 해야 할 일을 알려주는 배경 컨텍스트, Human : 사용자 메세지, AI : AI가 응답한 내용을 보여주는 상세 메세지의 3가지 역할을 가진 메세지의 기능을 같이 사용해줘야합니다.

그리고,  한번에 여러 리스트를 통해 결과를 받거나, Function calling 기능 또한 똑같이 사용할 수 있습니다. 

In [8]:
from langchain.globals import set_llm_cache
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.cache import InMemoryCache
from langchain.schema import HumanMessage, SystemMessage

# 채팅 기능 # chat message
# System : AI에게 해야 할 일을 알려주는 배경 컨텍스트
# Human : 사용자 메세지
# AI : AI가 응답한 내용을 보여주는 상세 메세지
chat = ChatOpenAI(temperature=.7,
                  callbacks=([StreamingStdOutCallbackHandler()]),  # 콜백 기능 지원
                  streaming=True,
                  verbose=True,
                  openai_api_key=openai_api_key
                  )

# 배이직
response = chat([
    SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
    HumanMessage(content="I like tomatoes, what should I eat?")
])
print(response)


# 한번에 여러번 호출하기
message_list = [HumanMessage(content="고양이 이름 지어줘"),
                HumanMessage(content="개 이름 지어줘")]
batch = chat.generate([message_list]) # generate로 한번에 생성한다.
print(batch)


# Function calling 기능
output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],

     functions=[{
         "name": "get_current_weather",
         "description": "Get the current weather in a given location",
         "parameters": {
             "type": "object",
             "properties": {
                 "location": {
                     "type": "string",
                     "description": "The city and state, e.g. San Francisco, CA"
                 },
                 "unit": {
                     "type": "string",
                     "enum": ["celsius", "fahrenheit"]
                 }
             },
             "required": ["location"]
         }
     }
     ]
)

print(output)

You could have a caprese salad with fresh mozzarella and basil.content='You could have a caprese salad with fresh mozzarella and basil.'
1. 미야 (Miya)
2. 코코 (Coco)
3. 루나 (Luna)
4. 토리 (Tori)
5. 레오 (Leo)
6. 다이아 (Daiya)
7. 오렌지 (Orange)
8. 스타 (Star)
9. 체리 (Cherry)
10. 블루 (Blue)generations=[[ChatGenerationChunk(text='1. 미야 (Miya)\n2. 코코 (Coco)\n3. 루나 (Luna)\n4. 토리 (Tori)\n5. 레오 (Leo)\n6. 다이아 (Daiya)\n7. 오렌지 (Orange)\n8. 스타 (Star)\n9. 체리 (Cherry)\n10. 블루 (Blue)', generation_info={'finish_reason': 'stop'}, message=AIMessageChunk(content='1. 미야 (Miya)\n2. 코코 (Coco)\n3. 루나 (Luna)\n4. 토리 (Tori)\n5. 레오 (Leo)\n6. 다이아 (Daiya)\n7. 오렌지 (Orange)\n8. 스타 (Star)\n9. 체리 (Cherry)\n10. 블루 (Blue)'))]] llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('92466d2c-8880-4dc1-a582-f55a7a8d7d0f'))]
content='' additional_kwargs={'function_call': {'arguments': '{\n  "location": "Boston, MA"\n}', 'name': 'get_current_weather'}}


### In memory

인메모리 기능은 전역변수 메모리 캐시를 저장해서 사용하는 기능이 있습니다. llm 모델에 사용해서 cache를 True False를 통해 선언해서 필요에 따라 사용할 수 있습니다. 이미 한번 생성한 질문에 대해서 똑같은 답변을 바로 답변해줍니다. 

In [43]:
import time 

# 전역변수에 메모리 캐시
chat = ChatOpenAI(temperature=.7,
                  callbacks=([StreamingStdOutCallbackHandler()]),  # 콜백 기능 지원
                  streaming=True,
                  verbose=True,
                  openai_api_key=openai_api_key
                  )

set_llm_cache(InMemoryCache())
start = time.time()
print(chat.generate([[HumanMessage(content="고양이 이름 지어줘")]]))
end = time.time()
print(end-start) # 2초

start = time.time()
print(chat.generate([[HumanMessage(content="고양이 이름 지어줘")]]))
end = time.time()
print(end-start) # 0.0019991397857666016초

고양이 이름을 지어드릴게요! 아래에 몇 가지 이름을 제안해 드릴게요:

1. 미야
2. 토리
3. 코코
4. 루나
5. 레오
6. 미로
7. 나비
8. 모카
9. 쿠키
10. 티니

이 중에서 마음에 드는 이름이 있으신가요? 혹은 다른 스타일의 이름을 원하시면 알려주세요!generations=[[ChatGenerationChunk(text='고양이 이름을 지어드릴게요! 아래에 몇 가지 이름을 제안해 드릴게요:\n\n1. 미야\n2. 토리\n3. 코코\n4. 루나\n5. 레오\n6. 미로\n7. 나비\n8. 모카\n9. 쿠키\n10. 티니\n\n이 중에서 마음에 드는 이름이 있으신가요? 혹은 다른 스타일의 이름을 원하시면 알려주세요!', generation_info={'finish_reason': 'stop'}, message=AIMessageChunk(content='고양이 이름을 지어드릴게요! 아래에 몇 가지 이름을 제안해 드릴게요:\n\n1. 미야\n2. 토리\n3. 코코\n4. 루나\n5. 레오\n6. 미로\n7. 나비\n8. 모카\n9. 쿠키\n10. 티니\n\n이 중에서 마음에 드는 이름이 있으신가요? 혹은 다른 스타일의 이름을 원하시면 알려주세요!'))]] llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('64dc3d35-3b30-4e38-8024-4055184e3689'))]
3.1634061336517334
generations=[[ChatGenerationChunk(text='고양이 이름을 지어드릴게요! 아래에 몇 가지 이름을 제안해 드릴게요:\n\n1. 미야\n2. 토리\n3. 코코\n4. 루나\n5. 레오\n6. 미로\n7. 나비\n8. 모카\n9. 쿠키\n10. 티니\n\n이 중에서 마음에 드는 이름이 있으신가요? 혹은 다른 스타일의 이름을 원하시면 알려주세요!', generation_info={'finish_reason': 'sto

In [44]:
chat = ChatOpenAI(temperature=.7,
                  callbacks=([StreamingStdOutCallbackHandler()]),  # 콜백 기능 지원
                  streaming=True,
                  verbose=True,
                  openai_api_key=openai_api_key,
                  cache=False # False로 사용
                  )

set_llm_cache(InMemoryCache())
start = time.time()
print(chat.generate([[HumanMessage(content="고양이 이름 지어줘")]]))
end = time.time()
print(end - start)  # 2초

start = time.time()
print(chat.generate([[HumanMessage(content="고양이 이름 지어줘")]]))
end = time.time()
print(end - start)  # 2초

1. 민트
2. 코코
3. 망고
4. 라임
5. 피치
6. 레오
7. 루나
8. 쿠키
9. 시나몬
10. 콩이generations=[[ChatGenerationChunk(text='1. 민트\n2. 코코\n3. 망고\n4. 라임\n5. 피치\n6. 레오\n7. 루나\n8. 쿠키\n9. 시나몬\n10. 콩이', generation_info={'finish_reason': 'stop'}, message=AIMessageChunk(content='1. 민트\n2. 코코\n3. 망고\n4. 라임\n5. 피치\n6. 레오\n7. 루나\n8. 쿠키\n9. 시나몬\n10. 콩이'))]] llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('01d502bd-4e24-49a9-afa6-408efc330059'))]
1.4914286136627197
1. 미야 (Miya)
2. 코코 (Coco)
3. 루나 (Luna)
4. 토리 (Tori)
5. 찰리 (Charlie)
6. 레오 (Leo)
7. 나비 (Nabi)
8. 샤론 (Sharon)
9. 오렌지 (Orange)
10. 피터 (Peter)generations=[[ChatGenerationChunk(text='1. 미야 (Miya)\n2. 코코 (Coco)\n3. 루나 (Luna)\n4. 토리 (Tori)\n5. 찰리 (Charlie)\n6. 레오 (Leo)\n7. 나비 (Nabi)\n8. 샤론 (Sharon)\n9. 오렌지 (Orange)\n10. 피터 (Peter)', generation_info={'finish_reason': 'stop'}, message=AIMessageChunk(content='1. 미야 (Miya)\n2. 코코 (Coco)\n3. 루나 (Luna)\n4. 토리 (Tori)\n5. 찰리 (Charlie)\n6. 레오 (Leo)\n7. 나비 (Nabi)\n8. 샤론 (Sharon)\n9. 오렌지 (

### embedding

임베딩을 사용하는 기능이 있습니다. 임베딩을 바로 사용하기기 보다는 Vector index를 만들기 위해 전달하는 인자로 많이 사용되고 주로 OpenAI의 임베딩을 많이 사용하지만, 비용적 절감을 위해 허깅페이스에서 제공하는 임베딩을 사용해서 절약하는 방식도 추천됩니다. 

In [16]:

from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=openai_api_key)

text = "안녕하세요! 해변에 갈 시간입니다"
text_embedding = embeddings.embed_query(text)

print (f"임베딩 길이 : {len(text_embedding)}")
print (f"샘플은 다움과 같습니다 : {text_embedding[:5]}...")

# 다중 문서 임베딩
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=openai_api_key)
text_embedding = embeddings.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
print(f"임베딩 길이 : {len(text_embedding)}, {len(text_embedding[0])}")
print(f"샘플은 다움과 같습니다 : {text_embedding[0][:5]}...")


# 허깅페이스 모델 센텐스 트랜스포머 임베딩 # pip install sentence_transformers
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}, # 모델의 전달할 키워드 인수
    # encode_kwargs={'normalize_embeddings': False},  # 모델의 `encode` 메서드를 호출할 때 전달할 키워드 인수
)
text = "안녕하세요! 해변에 갈 시간입니다"
text_embedding = hf_embeddings.embed_query(text)
print (f"임베딩 길이 : {len(text_embedding)}")
print (f"샘플은 다움과 같습니다 : {text_embedding[:5]}...")

임베딩 길이 : 1536
샘플은 다움과 같습니다 : [0.0062775725893334435, -0.02553328215339524, -0.01215778989813698, -0.013727984797761118, -0.02054711352729209]...
임베딩 길이 : 5, 1536
샘플은 다움과 같습니다 : [-0.020262643931117454, -0.006984279861728337, -0.022630838723440946, -0.02634143617913019, -0.03697932214749123]...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

임베딩 길이 : 768
샘플은 다움과 같습니다 : [0.011741544120013714, -0.031705114990472794, -0.013708159327507019, 0.013744891621172428, 0.04403568059206009]...


### prompt template

프롬프트 템플릿을 정의하고 사용이 가능합니다. 입력변수가 없는 프롬프트부터 여러개 입력변수를 사용하는 프롬프트까지 여러가지 사용할 수 있습니다. 
템플릿을 정의하고 {}를 통해 variable이 되는 인자를 정해주면 format을 통해 계속 계속 사용 할 수 있습니다.

In [17]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# 입력 변수가 없는 프롬프트 예제
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
prompt = no_input_prompt.format()
print(prompt)


# 하나의 입력 변수가 있는 예제 프롬프트
one_input_prompt = PromptTemplate(template="Tell me a {adjective} joke.", input_variables=["adjective"],)
prompt = one_input_prompt.format(adjective="funny")
print(prompt)


# 여러 입력 변수가 있는 프롬프트 예제
multiple_input_prompt = PromptTemplate(template="Tell me a {adjective} joke about {content}.",
                                       input_variables=["adjective", "content"],
                                       )
prompt = multiple_input_prompt.format(adjective="funny", content="chickens")
print(prompt)


# input_variables를 지정 안한 프롬프트 예제
no_variable_prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt = no_variable_prompt.format(product="colorful socks")
print(prompt)


# ChatPrompt 예제
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{human_text}"),
])

message = chat_prompt.format_messages(input_language="English",
                                      output_language="French",
                                      human_text="I love programming.")
print(message)


Tell me a joke.
Tell me a funny joke.
Tell me a funny joke about chickens.
What is a good name for a company that makes colorful socks?
[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


### Output Parser

일반적으로 LLM은 텍스트를 출력합니다. 하지만 보다 구조화된 정보를 얻고 싶을 수 있습니다.
이런 경우 출력 파서를 이용하여 LLM 응답을 구조화할 수 있습니다.
출력 파서는 두 가지 컨셉을 갖고 있습니다.

- Format instructions : 원하는 결과의 포멧을 지정하여 LLM에 알려줍니다.
- Parser : 원하는 텍스트 출력 구조 (보통 json) 을 추출하도록 합니다.

이 출력 구문 분석기를 사용하면 사용자가 임의의 JSON 스키마를 지정하고 해당 스키마를 준수하는 JSON 출력에 대해 LLM을 쿼리할 수 있습니다.

대규모 언어 모델은 누수가 있는 추상화라는 점에 유의하세요! 
올바른 형식의 JSON을 생성할 수 있는 충분한 용량을 갖춘 LLM을 사용해야 합니다. 
OpenAI 제품군에서 다빈치는 안정적으로 처리할 수 있지만, 퀴리는 이미 성능이 급격히 떨어집니다.
Pydantic을 사용하여 데이터 모델을 선언하세요. Pydantic의 BaseModel은 Python 데이터 클래스와 비슷하지만, 실제 유형 검사 + 강제성이 있습니다.

In [19]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import BaseOutputParser
from langchain.output_parsers import (PydanticOutputParser,
                                      OutputFixingParser,
                                      RetryWithErrorOutputParser,
                                      CommaSeparatedListOutputParser,
                                      )
from pydantic import BaseModel, Field, validator

In [20]:
"""
CommaSeparatedListOutputParser

LLM에게 명시적으로 get_format_instructions을 통해서 원하는 결과의 포멧을 지정하여 LLM에 알려줍니다.
아래 구문이 쿼리문으로 전달 됩니다. 그 만큼 토큰이 소비됩니다.
"Your response should be a list of comma separated values, "
"eg: `foo, bar, baz`"
"""

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = OpenAI(temperature=0, openai_api_key=openai_api_key)
_input = prompt.format(subject="ice cream flavors") # 'List five ice cream flavors.\nYour response should be a list of comma separated values, eg: `foo, bar, baz`'
output = model(_input)
print(output_parser.parse(output))

['Vanilla', 'Chocolate', 'Strawberry', 'Mint Chocolate Chip', 'Cookies and Cream']


In [21]:
"""
JSON parser / Function calling과 유사하게 사용할 수 있습니다.

Pydantic을 사용하면 사용자 지정 유효성 검사 로직을 쉽게 추가할 수 있습니다.
Pydantic의 BaseModel은 Python 데이터 클래스와 비슷하지만, 실제 유형 검사 + 강제성이 있습니다.
"""
    
class Joke(BaseModel):

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


"""
LLM에게 명시적으로 get_format_instructions을 통해서 원하는 결과의 포멧을 지정하여 LLM에 알려줍니다.
아래 구문이 쿼리문으로 전달 됩니다. 그 만큼 토큰이 소비됩니다.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output schema:
```
{schema}
```
"""
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

model = OpenAI(temperature=0,
               callbacks=([StreamingStdOutCallbackHandler()]),
               streaming=True ,
               verbose=True,
               openai_api_key=openai_api_key)

prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})

print(output)


{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}
{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}


In [22]:
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")
bad_response = '{"action": "search"}'
# parser.parse(bad_response)

"""
parser.parse(bad_response) 실행한다면 오류 action_input이 없기 때문에 에러가 발생한다. 

langchain.schema.output_parser.OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
field required (type=value_error.missing)
"""

model = OpenAI(temperature=0, openai_api_key=openai_api_key)

# Auto-Fixing Parser 활용
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
output = fix_parser.parse(bad_response)
print(output)

#대신, 프롬프트 (원래 출력뿐만 아니라)를 통과하는 RetryOutputParser를 사용하여 더 나은 응답을 얻기 위해 다시 시도 할 수 있습니다.
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)
output = retry_parser.parse_with_prompt(bad_response, prompt_value)
print(output)

action='search' action_input=''
action='search' action_input='who is leo di caprios gf?'


### Custom output parser
Custom으로 parser를 정의하고 싶으면 BaseOutputParser를 상속받아서 정의하면 됩니다. 
간단히는 parser 함수만 새롭게 정의해서 사용하면 자신만의 커스텀 아웃풋 파서를 생성할 수 있습니다. 
그외에도 get_format_instructions를 정의해서 string 타입으로 return한다면 프롬프트에 담아서 전달할 수 있습니다.

In [23]:
class CustomSpaceSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(" ")



parser = CustomSpaceSeparatedListOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n\n{query}\n",
    input_variables=["query"],
)

model = OpenAI(temperature=0,
               callbacks=([StreamingStdOutCallbackHandler()]),
               streaming=True ,
               verbose=True,
               openai_api_key=openai_api_key)

prompt_and_model = prompt | model | parser

output = prompt_and_model.invoke({"query": "Tell me a joke."})
print(output)


Q: What did the fish say when it hit the wall?
A: Dam!['Q:', 'What', 'did', 'the', 'fish', 'say', 'when', 'it', 'hit', 'the', 'wall?\nA:', 'Dam!']


### Link Chain

가장 일반적이고 귀중한 구성은 다음과 같습니다:

PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

구축 한 거의 모든 체인이이 빌딩 블록을 사용합니다.

체인과 체인을 연결 시킬 수 있고, 다양하게 사용 가능합니다. 

In [26]:
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["question"],
    template="""Q: {question}\nA:"""
)

# LLMChain 생성
llm_chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True
)

# LLMChain 실행
question = "기타를 잘 치는 방법은?"
print(llm_chain.predict(question=question))



> Entering new LLMChain chain...
Prompt after formatting:
Q: 기타를 잘 치는 방법은?
A:
 기타를 잘 치는 방법은 다음과 같습니다.

1. 손가락 위치를 잘 잡아야 합니다. 기타는 손가락의 위치에 따라 소리가 달라집니다.

2. 손가락을 잘 움직여야 합니다. 손가락을 잘 움직이면 소리가 더 잘 납니다.

3. 손가락의 압력
> Finished chain.
 기타를 잘 치는 방법은 다음과 같습니다.

1. 손가락 위치를 잘 잡아야 합니다. 기타는 손가락의 위치에 따라 소리가 달라집니다.

2. 손가락을 잘 움직여야 합니다. 손가락을 잘 움직이면 소리가 더 잘 납니다.

3. 손가락의 압력


In [27]:
# 랭체인 식 언어(LCEL) 
# 랭체인 식 언어(LangChain Expression Language)는 체인을 구성하고 스트리밍, 배치 및 비동기 지원을 기본적으로 제공하는 선언적 방법이다. 
# LCEL은 랭체인을 더 쉽게 사용할 수 있게 해준다.
# |를 통해서 간단히 연결 할 수 있고, 차이점으로는 invoke를 사용한다.

prompt = PromptTemplate(
    input_variables=["question"],
    template="""Q: {question}\nA:"""
)
question = "기타를 잘 치는 방법은?"
parser = CommaSeparatedListOutputParser()
chain = prompt | model | parser
output = chain.invoke({"question": question})
print(output)

['기타를 잘 치는 방법은 다음과 같습니다.\n\n1. 손가락 위치를 잘 잡아야 합니다. 기타는 손가락의 위치에 따라 소리가 달라집니다.\n\n2. 손가락을 잘 움직여야 합니다. 손가락을 잘 움직이면 소리가 더 잘 납니다.\n\n3. 손가락의 압력']


In [28]:
# SimpleSequentialChain

template1 = """당신은 극작가입니다. 연극 제목이 주어졌을 때, 그 줄거리를 작성하는 것이 당신의 임무입니다.

제목:{title}
시놉시스:"""
prompt1 = PromptTemplate(input_variables=["title"], template=template1)
chain1 = LLMChain(llm=model, prompt=prompt1)

template2 = """당신은 연극 평론가입니다. 연극의 시놉시스가 주어지면 그 리뷰를 작성하는 것이 당신의 임무입니다.

시놉시스:
{synopsis}
리뷰:"""
prompt2 = PromptTemplate(input_variables=["synopsis"], template=template2)
chain2 = LLMChain(llm=model,prompt=prompt2)

# SimpleSequentialChain으로 두 개의 체인을 연결
overall_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    verbose=True
)
print(overall_chain("서울 랩소디"))



> Entering new SimpleSequentialChain chain...

서울 랩소디는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 그는 자신의 음악을 통해 서울의 밤을 밝히고자 하는데, 그는 이를
서울 랩소디는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 그는 자신의 음악을 통해 서울의 밤을 밝히고자 하는데, 그는 이를


"서울 랩소디"는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 이 연극은 서울의 밤을 배경으로 하는 이야기를 다루고 있으며, 주

"서울 랩소디"는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 이 연극은 서울의 밤을 배경으로 하는 이야기를 다루고 있으며, 주

> Finished chain.
{'input': '서울 랩소디', 'output': '\n\n"서울 랩소디"는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 이 연극은 서울의 밤을 배경으로 하는 이야기를 다루고 있으며, 주'}


In [29]:
template1 = """당신은 극작가입니다. 연극 제목이 주어졌을 때, 그 줄거리를 작성하는 것이 당신의 임무입니다.

제목:{title}
시놉시스:"""
prompt1 = PromptTemplate(input_variables=["title"], template=template1)
chain1 = LLMChain(llm=model, prompt=prompt1, output_key="synopsis")

template2 = """당신은 연극 평론가입니다. 연극의 시놉시스가 주어지면 그 리뷰를 작성하는 것이 당신의 임무입니다.

시놉시스:
{synopsis}
리뷰:"""
prompt2 = PromptTemplate(input_variables=["synopsis"], template=template2)
chain2 = LLMChain(llm=model, prompt=prompt2, output_key="review")

overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["title"],
    output_variables=["synopsis", "review"],
    verbose=True
)
print(overall_chain("서울 랩소디"))



> Entering new SequentialChain chain...

> Finished chain.
{'title': '서울 랩소디', 'synopsis': '\n서울 랩소디는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 그는 자신의 음악을 통해 서울의 밤을 밝히고자 하는데, 그는 이를', 'review': '\n\n"서울 랩소디"는 서울의 밤을 배경으로 하는 이야기입니다. 주인공은 서울의 랩 씬에서 일하는 랩퍼로, 그는 자신의 음악을 통해 서울의 밤을 밝히고자 합니다. 이 연극은 서울의 밤을 배경으로 하는 이야기를 다루고 있으며, 주'}


### Text Split

텍스트를 청크단위로 나눌 수 있는 기능입니다. 글자 단위로 자를 수 있고, 원하는 토크나이저의 맞춰서 크기로 자를 수 있습니다. 그외 HTML과 같은 문서 또한 태그 단위로 나눌 수 있습니다. 

In [30]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import HTMLHeaderTextSplitter
from transformers import GPT2TokenizerFast

with open('/kaggle/input/langchain-tutorial/akazukin_all.txt', encoding='utf-8') as f:
    akazukin_all = f.read()


In [31]:

# separator로 자르고 글자 수에 맞춰서 자른다.
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 20,
    chunk_overlap  = 2,
    length_function = len,
    is_separator_regex = False,
)

texts = text_splitter.split_text(akazukin_all)
print(texts)


# "\n\n", "\n", " ", ""을 알아서 찾아서 리컬시브하게 알아서, 나눠서 글수 단위로 자른다.
text_splitter = RecursiveCharacterTextSplitter(
    # separator="\n\n" 따로 지정해줄 필요가 없다.
    chunk_size = 20,
    chunk_overlap  = 2,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_text(akazukin_all)
print(texts)


# 토큰 베이스 스플릿
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=20, chunk_overlap=2)
texts = text_splitter.split_text(akazukin_all)
print(texts)


# HTML 스플릿
html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

# 헤드만 따고 싶다.
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on) # pip install lxml
html_header_splits = html_splitter.split_text(html_string)
print(html_header_splits)

["제목: '전뇌 빨간 망토'", '제1장: 데이터 프론트', '밤이 되면 반짝이는 네오 도쿄. 고층 빌딩이 늘어서고, 네온사인이 거리를 수놓는다. 그 거리에서 빨간 두건을 쓴 소녀 미코는 불법 데이터 카우리아를 운반하는 배달원으로 일하고 있었다. 그녀는 어머니가 병에 걸려 치료비를 벌기 위해 데이터카우리아에 몸을 던지고 있었다.', "그러던 어느 날, 미코는 중요한 데이터를 운반하는 임무를 맡게 된다. 그 데이터에는 거대 기업 '울프 코퍼레이션'의 시민에 대한 악랄한 지배를 폭로하는 정보가 담겨 있었다. 그녀는 데이터를 받아 목적지로 향한다.", '제2장: 울프 코퍼레이션의 함정', "미코는 목적지인 술집 '할머니의 집'으로 향하는 길에 울프 코퍼레이션의 요원들에게 쫓기게 된다. 그들은 '빨간 망토'라는 데이터 카우리아에 대한 소문을 듣고 데이터를 탈취하려 했다. 미코는 교묘하게 요원들을 흩뿌리고 술집에 도착한다.", '제3장: 배신과 재회', "술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기다리고 있었다. 료는 그녀의 어릴 적 친구이자 그 역시 울프 코퍼레이션과 싸우는 해커 집단의 일원이었다. 하지만 료는 미코에게 배신감을 느꼈고, 그녀가 데이터 카우리아에 몸을 던진 것에 화가 났다.", '그럼에도 불구하고 미코는 료에게 데이터를 건네며 울프 코퍼레이션에 대한 반격을 믿기로 한다. 두 사람은 함께 울프 코퍼레이션의 음모를 밝혀내고 시민들을 구하기로 결심한다.', '제4장: 울프 코퍼레이션의 붕괴', '미코와 료는 해커 집단과 함께 울프 코퍼레이션에 대한 최후의 결전을 벌인다. 능숙한 해킹 기술과 신체 능력으로 그들은 울프 코퍼레이션의 보안을 차례로 뚫어 나간다. 그 과정에서 미코는 울프 코퍼레이션이 어머니의 병에 관여하고 있다는 사실을 알게 된다. 그녀는 분노에 휩싸여 울프코퍼레이션에 대한 복수를 다짐한다.', '제5장: 결전의 순간', '미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는

["제목: '전뇌 빨간 망토'", '제1장: 데이터 프론트', '밤이 되면 반짝이는 네오 도쿄. 고층 빌딩이 늘어서고, 네온사인이 거리를 수놓는다. 그 거리에서 빨간 두건을 쓴 소녀 미코는 불법 데이터 카우리아를 운반하는 배달원으로 일하고 있었다. 그녀는 어머니가 병에 걸려 치료비를 벌기 위해 데이터카우리아에 몸을 던지고 있었다.', "그러던 어느 날, 미코는 중요한 데이터를 운반하는 임무를 맡게 된다. 그 데이터에는 거대 기업 '울프 코퍼레이션'의 시민에 대한 악랄한 지배를 폭로하는 정보가 담겨 있었다. 그녀는 데이터를 받아 목적지로 향한다.", '제2장: 울프 코퍼레이션의 함정', "미코는 목적지인 술집 '할머니의 집'으로 향하는 길에 울프 코퍼레이션의 요원들에게 쫓기게 된다. 그들은 '빨간 망토'라는 데이터 카우리아에 대한 소문을 듣고 데이터를 탈취하려 했다. 미코는 교묘하게 요원들을 흩뿌리고 술집에 도착한다.", '제3장: 배신과 재회', "술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기다리고 있었다. 료는 그녀의 어릴 적 친구이자 그 역시 울프 코퍼레이션과 싸우는 해커 집단의 일원이었다. 하지만 료는 미코에게 배신감을 느꼈고, 그녀가 데이터 카우리아에 몸을 던진 것에 화가 났다.", '그럼에도 불구하고 미코는 료에게 데이터를 건네며 울프 코퍼레이션에 대한 반격을 믿기로 한다. 두 사람은 함께 울프 코퍼레이션의 음모를 밝혀내고 시민들을 구하기로 결심한다.', '제4장: 울프 코퍼레이션의 붕괴', '미코와 료는 해커 집단과 함께 울프 코퍼레이션에 대한 최후의 결전을 벌인다. 능숙한 해킹 기술과 신체 능력으로 그들은 울프 코퍼레이션의 보안을 차례로 뚫어 나간다. 그 과정에서 미코는 울프 코퍼레이션이 어머니의 병에 관여하고 있다는 사실을 알게 된다. 그녀는 분노에 휩싸여 울프코퍼레이션에 대한 복수를 다짐한다.', '제5장: 결전의 순간', '미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는

### Document

문서를 만들 수 있습니다. 직접 document를 이용해서 문서를 만들 수 있고, 혹은 다른 타입의 파일을 불러와서 문서를 자동으로 생성 할 수 있습니다. 이경우 앞에서 사용한 split을 같이 사용하게 됩니다. 이렇게 문서를 만들면 나중에 vector index에서 사용하기 용이해집니다. 

In [38]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader, DataFrameLoader, BSHTMLLoader
from langchain.schema import Document


# document 만들기
my_page = Document(
page_content="이 문서는 제 문서입니다. 다른 곳에서 수집한 텍스트로 가득합니다.",
metadata={'explain': 'The LangChain Papers'})
print(my_page)


# 다중 문서 만들기
my_list = [
"Hi there!",
"Oh, hello!",
"What's your name?",
"My friends call me World",
"Hello World!"
]

my_pages = [Document(page_content = i) for i in my_list]
print(my_pages)

page_content='이 문서는 제 문서입니다. 다른 곳에서 수집한 텍스트로 가득합니다.' metadata={'explain': 'The LangChain Papers'}
[Document(page_content='Hi there!'), Document(page_content='Oh, hello!'), Document(page_content="What's your name?"), Document(page_content='My friends call me World'), Document(page_content='Hello World!')]


In [42]:
# PyPDF Loader # pip install pypdf

loader = PyPDFLoader("/kaggle/input/langchain-tutorial/field-guide-to-data-science.pdf")
pages = loader.load_and_split()
print(pages[:2])

[Document(page_content='DATA SCIENCEtoTHE\nFIEL D GUIDE\n    \n \nSECOND  \nEDITION\n© COPYRIGHT 2015 BOOZ ALLEN HAMILTON INC. ALL RIGHTS RESERVED.', metadata={'source': '/kaggle/input/langchain-tutorial/field-guide-to-data-science.pdf', 'page': 1}), Document(page_content='FOREWORD\nData Science touches every aspect of our lives on a \ndaily basis. When we visit the doctor, drive our cars, \nget on an airplane, or shop for services, Data Science \nis changing the way we interact with and explore  \nour world.  \nOur world is now measured, \nmapped, and recorded in digital \nbits. Entire lives, from birth to \ndeath, are now catalogued in \nthe digital realm. These data, \noriginating from such diverse \nsources as connected vehicles, \nunderwater microscopic cameras, \nand photos we post to social \nmedia, have propelled us into \nthe greatest age of discovery \nhumanity has ever known. It is \nthrough Data Science that we \nare unlocking the secrets hidden \nwithin these data. We are 

In [40]:
# DataFrame Loader
df = pd.read_csv("/kaggle/input/langchain-tutorial/mlb_teams_2012.csv")
loader = DataFrameLoader(df, page_content_column="Team")
pages = loader.load_and_split()
print(pages[:5])

[Document(page_content='Nationals', metadata={' "Payroll (millions)"': 81.34, ' "Wins"': 98}), Document(page_content='Reds', metadata={' "Payroll (millions)"': 82.2, ' "Wins"': 97}), Document(page_content='Yankees', metadata={' "Payroll (millions)"': 197.96, ' "Wins"': 95}), Document(page_content='Giants', metadata={' "Payroll (millions)"': 117.62, ' "Wins"': 94}), Document(page_content='Braves', metadata={' "Payroll (millions)"': 83.31, ' "Wins"': 94})]


In [41]:
# BS4 HTML Loader
loader = BSHTMLLoader("/kaggle/input/langchain-tutorial/fake-content.html")
pages = loader.load_and_split()
print(pages[:5])

[Document(page_content='Test Title\n\n\nMy First Heading\nMy first paragraph.', metadata={'source': '/kaggle/input/langchain-tutorial/fake-content.html', 'title': 'Test Title'})]


### vector index

벡터 인덱스는 RAG에서 가장 기본인 되는 요소 입니다. 쿼리와 가장 유사한 문장을 찾아서 보다 정확한 답변을 얻을 수 있게 찾는 기능입니다. 여기서는 Faiss라는 라이브러리를 이용해서 사용하는 예제를 보여줍니다. 메타데이터를 통해 보다 문서가 어떤 내용을 갖고 있는지 추가적인 정보를 전달 해줄 수 있습니다. 이러면 보다 정확한 문서를 찾게될 수 있습니다. similarity_search를 통해 코사인 유사도를 비교하고 k개 만큼의 문서를 사용합니다. 

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

# 데이터 준비
with open('/kaggle/input/langchain-tutorial/akazukin_all.txt', encoding='utf-8') as f:
    akazukin_all = f.read()

# 청크 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 청크의 최대 문자 수
    chunk_overlap=20,  # 최대 오버랩 문자 수
)
texts = text_splitter.split_text(akazukin_all)

# 확인
print(len(texts))
for text in texts:
    print(text[:10], ":", len(text))

# 메타데이터 준비
metadatas = [
    {"source": "1장"},
    {"source": "2장"},
    {"source": "3장"},
    {"source": "4장"},
    {"source": "5~6장"},
    {"source": "7장"}
]

# Faiss 벡터 인덱스 생성
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
docsearch = FAISS.from_texts(texts=texts,  # 청크 배열
                             embedding=embeddings,  # 임베딩
                             metadatas=metadatas  # 메타데이터
                             )

query="미코의 소꿉친구 이름은?"
docs = docsearch.similarity_search(query, k=3)

for i in docs:
    print(i.page_content)


6
제목: '전뇌 빨간 : 299
제2장: 울프 코퍼 : 162
제3장: 배신과 재 : 273
제4장: 울프 코퍼 : 206
제5장: 결전의 순 : 294
제7장: 새로운 시 : 195
제2장: 울프 코퍼레이션의 함정

미코는 목적지인 술집 '할머니의 집'으로 향하는 길에 울프 코퍼레이션의 요원들에게 쫓기게 된다. 그들은 '빨간 망토'라는 데이터 카우리아에 대한 소문을 듣고 데이터를 탈취하려 했다. 미코는 교묘하게 요원들을 흩뿌리고 술집에 도착한다.

제3장: 배신과 재회
제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는 시민을 지배하려는 사악한 야망을 드러내며 자신의 압도적인 힘을 과시한다. 하지만 미코와 료는 서로를 도와가며 울프 박사와 싸우고 그의 약점을 찾아낸다.

제6장: 진실의 해방

미코는 울프 박사의 약점을 파고들어 그를 쓰러뜨리는데 성공한다. 그리고 해커 집단과 함께 울프 코퍼레이션의 악행을 세상에 공개하고 시민들을 해방시킨다. 이 승리로 미코의 어머니의 치료법도 찾아내고, 그녀의 병은 완치된다.
제7장: 새로운 시작

울프 코퍼레이션이 무너진 후, 미코와 료는 서로의 과거를 용서하고 다시 우정을 회복한다. 미코는 데이터카우리아를 그만두고 료와 함께 새로운 길을 걷기 시작한다. 그들은 스스로의 힘으로 미래의 네오 도쿄를 더 나은 도시로 바꾸어 나갈 것을 다짐한다. 이것은 미코와 료, 그리고 전뇌 빨간 망토의 새로운 모험의 시작이었다.

결말


# Part.2에서 계속 